__собираем предложения о земельных участках__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/land/avito' 
# cian_raw_data_path = 'data/raw/cian'
result_file_path = 'data/data_land.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/land/avito/avito_2022-09-08_16-05_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-12_13-51_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-13_13-56_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-15_10-31_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-16_15-10_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-19_12-39_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-22_09-57_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-26_11-22_raw.xlsx',
 'data/raw/land/avito/avito_2022-10-08_20-38_raw.xlsx',
 'data/raw/land/avito/avito_2022-10-14_15-54_raw.xlsx',
 'data/raw/land/avito/avito_2022-11-01_13-58_raw.xlsx',
 'data/raw/land/avito/avito_2022-11-04_17-12_raw.xlsx',
 'data/raw/land/avito/avito_2022-11-07_10-46_raw.xlsx']

In [7]:
# for f in raw_data_files:
#     df = pd.read_excel(f)
#     df['place']='sevastopol'
#     df.to_excel(f,index=False)
#     print(f)

In [8]:
data_avito = pd.concat([
        pd
        .read_excel(f)
        #.dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

print(len(data_avito))

14090


In [9]:
# data_avito

In [10]:
from lib.avito import AvitoDataCleanerRealtyLand

data_avito = AvitoDataCleanerRealtyLand().transform( data_avito )

print(len(data_avito))

14090


In [11]:
data_avito.sample(3)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,area,is_IJS,priceM,priceMU,area_size_category
3872,2543864418,"Участок 10,3 сот. (ИЖС)",6000000,,Муссонная ул.,"Продам участок общей площадью 10.3 сотки на ул. Муссонная, состоящий из двух смежных участков (5,8 соток и 4,4 соток) (р-н ул. Фадеева, Стрелецкая бухта). \nУчасток огорожен с 3-х сторон забором, \nКоммуникации: электроэнергия, городская вода, газ, городская",12,2022-09-15 10:31:26.039,sevastopol,10.30,True,6.00,0.58,8-20
7999,2304340705,Участок 8 сот. (ИЖС),6000000,,Почетная ул.,"Продается отличный участок Ижс 8 соток, ул. Почётная 33, размером 18м х 46м. Видовые характеристики, расположение, свежая застройка вокруг, всё это подчеркнёт ваш статус. \n\nДо остановки общественного транспорта 1,1км или 15 мин. Пешком. Севавтодор проклады",6,2022-09-26 11:22:23.724,sevastopol,8.00,True,6.00,0.75,4-8
12163,2336322834,Участок 3 сот. (ИЖС),2400000,,"Гагаринский муниципальный округ, микрорайон Правая Гераклея","Продается участок 3 сот. В Гагаринском районе, Правая Гераклея. \n\nУчасток находится в самом развитом районе г. Севастополь. Если вы ищите участок для строительства дома своей мечты, то это то что вам нужно. Тихое, спокойное, уютное место. Коммуникации: эле",2,2022-11-04 17:12:55.140,sevastopol,3.00,True,2.40,0.80,2-4


In [12]:
assert len(data_avito)>0

In [13]:
data_avito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14090 entries, 0 to 14089
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   avito_id            14090 non-null  int64         
 1   title               14090 non-null  object        
 2   price               14090 non-null  int64         
 3   obj_name            14090 non-null  object        
 4   adr                 14090 non-null  object        
 5   description         14090 non-null  object        
 6   avito_page          14090 non-null  int64         
 7   ts                  14090 non-null  datetime64[ns]
 8   place               14090 non-null  object        
 9   area                14090 non-null  float64       
 10  is_IJS              14090 non-null  bool          
 11  priceM              14090 non-null  float64       
 12  priceMU             14090 non-null  float64       
 13  area_size_category  14084 non-null  category  

---

In [14]:
data_avito['url'] = 'https://avito.ru/' + data_avito['avito_id'].astype(int).astype(str)
data_avito = data_avito.drop(columns=['avito_id'])
# data_cian = data_cian.rename(columns={'cian_url':'url'})

In [15]:
data = data_avito.reset_index(drop=True) 
print(len(data))
assert len(data)>0

14090


## обновляем таблицу адресов

In [16]:
# data_ = data.sample(15).reset_index(drop=True)
# data_[['adr']]

In [17]:
places = { 
# поле 'place' и название города
'bahchisaray': 'Бахчисарай,',
      'sudak': 'Судак,',
  'sevastopol': 'Севастополь,',
}

def norm_place(d,places=places):
    if not d['place'] in places: return d['adr_']
    p = places[ d['place'] ]
    return d['adr_'] if d['adr_'].find(p)>-1 else p+d['adr_']

In [18]:
from lib.locator import AddressTransformer

atr = AddressTransformer()
data['adr_'] = data['adr'].progress_apply(atr.transform)
data['adr_'] = data[['adr_','place']].progress_apply(norm_place,axis=1)
data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

  0%|          | 0/14090 [00:00<?, ?it/s]

  0%|          | 0/14090 [00:00<?, ?it/s]

In [19]:
data[['adr_orig','adr']].sample(10)

,adr_orig,adr
2160,Крепостное ш.,"Севастополь,Крепостное ш."
658,"Редутная ул., 25","Севастополь,Редутная ул., 25"
4158,"Севастополь, Рубежный проезд, 40","Севастополь, Рубежный проезд, 40"
10782,"Крепостное ш., 23к3","Севастополь,Крепостное ш., 23к3"
12344,с. Угловое,"Севастополь,с. Угловое"
10116,"Севастополь, 4-я Голландская улица, 5","Севастополь, 4-я Голландская улица, 5"
11885,садовое товарищество Строитель СУ-432,"Севастополь,садовое товарищество Строитель СУ-432"
12736,"дачный потребительский кооператив Горный, пер. Капитана Дьячкова","Севастополь,дачный потребительский кооператив Горный, пер. Капитана Дьячкова"
7836,садоводческое товарищество Усть-Бельбек,"Севастополь,садоводческое товарищество Усть-Бельбек"
2285,"ул. Бисти, 3","Севастополь,ул. Бисти, 3"


In [20]:
# from lib.locator import GeocoderSimpleOSM
from lib.locator import GeocoderYandex
from yandex_keys import keys

atr = AddressTransformer()
locator = GeocoderYandex(keys[0])
# locator = GeocoderSimpleOSM()

loc_file_path = 'data/location.pkl' # таблица адресов
loc = pd.read_pickle(loc_file_path)
# loc = pd.DataFrame([],columns=['adr','latitude','longitude',])
logging.info(f'LocationUpdater: {len(loc)} addresses in index')

# собираем все адреса в один список
loc = loc.merge(
        data[['adr']].drop_duplicates(),
        how='outer',
        on='adr',
    ).drop_duplicates().reset_index(drop=True)

logging.info(f'LocationUpdater: {len(loc)} addresses total')

# собираем все адреса с геопозицией
loc_def = loc[~loc['latitude'].isna()].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_def)} addresses defined')

# собираем все адреса без геопозиции
loc_undef = loc[loc['latitude'].isna()][['adr']].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_undef)} addresses undefined')

if len(loc_undef)>0:
    # определяем координаты по строке адреса
    loc_undef = loc_undef.join( loc_undef['adr'].progress_apply(locator.transform).apply(pd.Series) ).dropna()     
    logging.info(f'LocationUpdater: {len(loc_undef)} new addresses found')
    loc = pd.concat([loc_def,loc_undef]).reset_index(drop=True) 
    assert len(loc)>0
    loc.to_pickle(loc_file_path)

[INFO    ] 2022-11-07 12:12:18 | LocationUpdater: 7096 addresses in index
[INFO    ] 2022-11-07 12:12:18 | LocationUpdater: 7104 addresses total
[INFO    ] 2022-11-07 12:12:18 | LocationUpdater: 7096 addresses defined
[INFO    ] 2022-11-07 12:12:18 | LocationUpdater: 8 addresses undefined


  0%|          | 0/8 [00:00<?, ?it/s]

[INFO    ] 2022-11-07 12:12:21 | LocationUpdater: 8 new addresses found


In [21]:
print( len(loc) )
loc.sample(3)

7104


,latitude,longitude,adr
5050,44.540251,33.541063,"Севастополь,Античный б-р, 66"
2261,44.565501,33.408651,"Севастополь, район Гагаринский, Казачья Бухта мкр, улица Рубежная, 6"
1530,44.59,33.52,"Севастополь, район Ленинский, улица Супруна, 8"


## дополняем данные геометкой

In [22]:
# data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

data = data.merge(loc,on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isna() ] ) )

assert len(data)>0
data.to_pickle(result_file_path)

data.info()

всего записей: 14090
записей без геометки: 0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 14090 entries, 0 to 14089
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               14090 non-null  object        
 1   price               14090 non-null  int64         
 2   obj_name            14090 non-null  object        
 3   adr_orig            14090 non-null  object        
 4   description         14090 non-null  object        
 5   avito_page          14090 non-null  int64         
 6   ts                  14090 non-null  datetime64[ns]
 7   place               14090 non-null  object        
 8   area                14090 non-null  float64       
 9   is_IJS              14090 non-null  bool          
 10  priceM              14090 non-null  float64       
 11  priceMU             14090 non-null  float64       
 12  area_size_category  14084 non-null  category      
 13  u